In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

## STEP1: LOADING MNIST TRAIN DATASET

In [ ]:
train_dateset=datasets.MNIST('./data/', train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dateset=datasets.MNIST('./data/', train=False,
                            transform=transforms.ToTensor(),
                            download=True)

In [ ]:
print(train_dateset.data.shape, train_dateset.targets.shape)
print(test_dateset.data.shape, test_dateset.targets.shape)

## STEP2: MAKE DATASET ITERABLE

In [ ]:
batch_size=100
num_epochs=5

train_loader=DataLoader(dataset=train_dateset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader=DataLoader(dataset=test_dateset,
                        batch_size=batch_size,
                        shuffle=False)

## STEP3: CREATE MODEL CLASS

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim,
                 layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim=hidden_dim
        self.layer_dim=layer_dim
        self.rnn=nn.RNN(input_size=input_dim,
                        hidden_size=hidden_dim,
                        num_layers=layer_dim,
                        batch_first=True, 
                        nonlinearity='relu')
        self.fc=nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0=torch.zeros(self.layer_dim, x.shape[0],
                       self.hidden_dim).requires_grad_()
        out, hn=self.rnn(x, h0.detach())
        out=self.fc(out[:,-1,:])
        return out

## STEP4: INSTANITIAT MODEL CLASS

In [ ]:
input_dim=28
hidden_dim=100
layer_dim=1
output_dim=10
model=RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

## STEP5: INSTANTIATE LOSS CLASS

In [ ]:
criterion=nn.CrossEntropyLoss()

## STEP6: INTANTIATE OPTIMIZER CLASS

In [ ]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

## STEP7: TRAIN MODEL

In [ ]:
seq_dim=28
iter=0
for epoch in range(num_epochs):
    for images, labels in train_loader:
        model.train()
        images=images.view(-1, seq_dim, input_dim).requires_grad_()
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iter+=1
        
        if iter%500==0:
            model.eval()
            correct=0
            total=0
            for images, labels in test_loader:
                images=images.view(-1, seq_dim, input_dim)
                outputs=model(images)
                _,predicted=torch.max(outputs.data, 1)
                total+=labels.size(0)
                correct+=(predicted==labels).sum()
                
            accuracy=100*correct/total
            print("iter:{}. Loss:{}. Accuracy:{}".format(iter,
                                                        loss.item(),
                                                        accuracy))